In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath=os.path.sep.join([r'D:\Mask-Detection-and-Recognition-using-Deep-Learning-Keras-master\face_detector','deploy.prototxt'])
weightsPath=os.path.sep.join([r'D:\Mask-Detection-and-Recognition-using-Deep-Learning-Keras-master\face_detector','res10_300x300_ssd_iter_140000.caffemodel'])


In [3]:
prototxtPath

'D:\\Mask-Detection-and-Recognition-using-Deep-Learning-Keras-master\\face_detector\\deploy.prototxt'

In [4]:
weightsPath

'D:\\Mask-Detection-and-Recognition-using-Deep-Learning-Keras-master\\face_detector\\res10_300x300_ssd_iter_140000.caffemodel'

In [5]:
net = cv2.dnn.readNet(prototxtPath,weightsPath)

In [6]:
model=load_model(r'D:\Mask-Detection-and-Recognition-using-Deep-Learning-Keras-master\v.model')

In [7]:
image=cv2.imread(r'D:\Mask-Detection-and-Recognition-using-Deep-Learning-Keras-master\vishalll.jpg')

In [8]:
image

array([[[227, 210, 224],
        [226, 209, 223],
        [227, 210, 224],
        ...,
        [140, 224, 165],
        [141, 225, 166],
        [141, 224, 162]],

       [[227, 210, 224],
        [227, 210, 224],
        [229, 212, 226],
        ...,
        [135, 219, 160],
        [134, 218, 159],
        [133, 216, 154]],

       [[228, 211, 225],
        [228, 211, 225],
        [231, 214, 228],
        ...,
        [134, 218, 159],
        [132, 216, 157],
        [131, 216, 154]],

       ...,

       [[ 70, 105, 155],
        [ 73, 108, 158],
        [ 75, 113, 161],
        ...,
        [181, 211, 228],
        [186, 216, 233],
        [190, 220, 237]],

       [[ 58,  93, 143],
        [ 61,  96, 146],
        [ 63, 100, 150],
        ...,
        [182, 213, 228],
        [190, 221, 236],
        [196, 226, 243]],

       [[ 53,  88, 138],
        [ 53,  88, 138],
        [ 52,  89, 139],
        ...,
        [169, 200, 215],
        [180, 211, 226],
        [189, 219, 236]]

In [9]:
(h,w)=image.shape[:2]

In [10]:
(h,w)

(1872, 1405)

In [11]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [12]:
blob

array([[[[128., 128., 131., ...,  31.,  31.,  30.],
         [125., 126., 127., ...,  24.,  42.,  31.],
         [131., 130., 126., ...,  31.,  38.,  26.],
         ...,
         [ 20.,  21.,   6., ...,  93.,  95.,  95.],
         [ 17.,  31.,  19., ...,  87.,  84.,  84.],
         [-29., -17.,  16., ...,  82.,  78.,  78.]],

        [[ 38.,  38.,  41., ...,  47.,  43.,  41.],
         [ 35.,  36.,  37., ...,  35.,  53.,  46.],
         [ 41.,  40.,  37., ...,  42.,  49.,  40.],
         ...,
         [-13.,  -9., -23., ...,  46.,  47.,  48.],
         [-17.,   0., -10., ...,  40.,  37.,  37.],
         [-64., -48., -12., ...,  40.,  36.,  34.]],

        [[106., 106., 108., ...,  45.,  39.,  36.],
         [103., 104., 105., ...,  31.,  49.,  40.],
         [109., 108., 105., ...,  37.,  44.,  37.],
         ...,
         [ 83.,  89.,  78., ..., 112., 116., 116.],
         [ 82.,  99.,  88., ..., 110., 107., 108.],
         [ 38.,  51.,  85., ..., 109., 105., 105.]]]], dtype=float32)

In [13]:
blob.shape

(1, 3, 300, 300)

In [14]:
net.setInput(blob)
detections=net.forward()

In [15]:
detections

array([[[[0.        , 1.        , 0.99999964, ..., 0.13151532,
          0.6364422 , 0.4034794 ],
         [0.        , 1.        , 0.12952551, ..., 3.9960387 ,
          4.8366756 , 4.981924  ],
         [0.        , 1.        , 0.12041694, ..., 4.006214  ,
          0.8332665 , 4.9882526 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [16]:
#loop over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        #label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()